# MBA Module 4 Project 1,2 and 3 has been combined

### Initializing Spark Environment

In [2]:
# Importing all the packages and setting up environment

import os
import sys

spark_path = 'D:\\spark-2.4.3-bin-hadoop2.7'
os.environ['SPARK_HOME']= spark_path
os.environ['HADOOP_HOME']=spark_path
sys.path.append(spark_path+'/bin')
sys.path.append(spark_path+'/python')
sys.path.append(spark_path+'/python/pyspark')
sys.path.append(spark_path+'/python/lib')
sys.path.append(spark_path+'/python/lib/pyspark.zip')
sys.path.append(spark_path+'/python/lib/py4j-0.10.4-src.zip')

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('tree').getOrCreate()
# Reading the data
df = spark.read.csv('D:\\ashu\\REVA\\MBA\\Mod4\\preprocessed_13.csv', inferSchema=True, header=True)
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Dur: double (nullable = true)
 |-- Proto: string (nullable = true)
 |-- SrcAddr: string (nullable = true)
 |-- Sport: string (nullable = true)
 |-- Dir: string (nullable = true)
 |-- DstAddr: string (nullable = true)
 |-- Dport: string (nullable = true)
 |-- State: string (nullable = true)
 |-- sTos: double (nullable = true)
 |-- TotPkts: integer (nullable = true)
 |-- TotBytes: integer (nullable = true)
 |-- SrcBytes: integer (nullable = true)
 |-- Label: string (nullable = true)
 |-- Attacked: integer (nullable = true)



In [4]:
df.describe()

DataFrame[summary: string, _c0: string, StartTime: string, Dur: string, Proto: string, SrcAddr: string, Sport: string, Dir: string, DstAddr: string, Dport: string, State: string, sTos: string, TotPkts: string, TotBytes: string, SrcBytes: string, Label: string, Attacked: string]

In [5]:
df.count()

1889141

In [7]:
df.dtypes

[('_c0', 'int'),
 ('StartTime', 'string'),
 ('Dur', 'double'),
 ('Proto', 'string'),
 ('SrcAddr', 'string'),
 ('Sport', 'string'),
 ('Dir', 'string'),
 ('DstAddr', 'string'),
 ('Dport', 'string'),
 ('State', 'string'),
 ('sTos', 'double'),
 ('TotPkts', 'int'),
 ('TotBytes', 'int'),
 ('SrcBytes', 'int'),
 ('Label', 'string'),
 ('Attacked', 'int')]

In [6]:
#display top 10 records
df.show(10)

+---+--------------------+------------------+-----+--------------+-----+-----+-------------+-----+-----+----+-------+--------+--------+--------------------+--------+
|_c0|           StartTime|               Dur|Proto|       SrcAddr|Sport|  Dir|      DstAddr|Dport|State|sTos|TotPkts|TotBytes|SrcBytes|               Label|Attacked|
+---+--------------------+------------------+-----+--------------+-----+-----+-------------+-----+-----+----+-------+--------+--------+--------------------+--------+
|  0|2011/08/15 17:13:...|          5.170088|  tcp|  2.157.248.25| 3146|   ->|147.32.84.118| 6881| S_RA| 0.0|      4|     252|     132|flow=Background-T...|       0|
|  1|2011/08/15 17:15:...| 6.609413000000001|  tcp|148.100.181.73|51668|   ->|147.32.84.118| 6881| S_RA| 0.0|      4|     252|     132|flow=Background-T...|       0|
|  2|2011/08/15 17:20:...|1.7971849999999998|  tcp|61.177.250.153| 7212|   ->| 147.32.84.40| 4899| S_RA| 0.0|      4|     244|     124|flow=Background-T...|       0|
|  3

In [14]:
# Convert target into numerical categories, not required in this case, as it is already numeric
# labelIndexer = StringIndexer(inputCol="Species", outputCol="label")

In [8]:
from pyspark.sql.types import IntegerType, DateType, DoubleType

In [10]:
df.take(2)

[Row(_c0=0, StartTime='2011/08/15 17:13:51.499832', Dur=5.170088, Proto='tcp', SrcAddr='2.157.248.25', Sport='3146', Dir='   ->', DstAddr='147.32.84.118', Dport='6881', State='S_RA', sTos=0.0, TotPkts=4, TotBytes=252, SrcBytes=132, Label='flow=Background-TCP-Attempt', Attacked=0),
 Row(_c0=1, StartTime='2011/08/15 17:15:33.511251', Dur=6.609413000000001, Proto='tcp', SrcAddr='148.100.181.73', Sport='51668', Dir='   ->', DstAddr='147.32.84.118', Dport='6881', State='S_RA', sTos=0.0, TotPkts=4, TotBytes=252, SrcBytes=132, Label='flow=Background-TCP-Attempt', Attacked=0)]

In [11]:
#consider only hour
df1 = df.withColumn("StartTime", df["StartTime"].substr(12,2))

In [12]:
df1.take(2)

[Row(_c0=0, StartTime='17', Dur=5.170088, Proto='tcp', SrcAddr='2.157.248.25', Sport='3146', Dir='   ->', DstAddr='147.32.84.118', Dport='6881', State='S_RA', sTos=0.0, TotPkts=4, TotBytes=252, SrcBytes=132, Label='flow=Background-TCP-Attempt', Attacked=0),
 Row(_c0=1, StartTime='17', Dur=6.609413000000001, Proto='tcp', SrcAddr='148.100.181.73', Sport='51668', Dir='   ->', DstAddr='147.32.84.118', Dport='6881', State='S_RA', sTos=0.0, TotPkts=4, TotBytes=252, SrcBytes=132, Label='flow=Background-TCP-Attempt', Attacked=0)]

In [13]:
#check for each protocol type, what was the count
df1.groupBy('Proto').count().orderBy('count').collect()

[Row(Proto='udt', count=7),
 Row(Proto='rtcp', count=1334),
 Row(Proto='rtp', count=1757),
 Row(Proto='icmp', count=14123),
 Row(Proto='tcp', count=359837),
 Row(Proto='udp', count=1512083)]

In [17]:
from pyspark.sql.functions import *

### Data preprocessing

In [22]:
#convert udt, rtcp and rtp into others
df2 = df1.withColumn("Proto", when(df["Proto"] == 'udt', 'others').otherwise(df1["Proto"]))
df3 = df2.withColumn("Proto", when(df2["Proto"] == 'rtcp', 'others').otherwise(df2["Proto"]))
df4 = df3.withColumn("Proto", when(df3["Proto"] == 'rtp', 'others').otherwise(df3["Proto"]))
df4.groupBy('Proto').count().orderBy('count').collect()

[Row(Proto='others', count=3098),
 Row(Proto='icmp', count=14123),
 Row(Proto='tcp', count=359837),
 Row(Proto='udp', count=1512083)]

In [22]:
#check different types of State
df.groupBy('State').count().orderBy('count').collect()

[Row(State='FRPA_R', count=1),
 Row(State='FSPAC_SRPA', count=1),
 Row(State='SRA_FSRA', count=1),
 Row(State='SPAC_FSRPA', count=1),
 Row(State='FSPA_FRA', count=1),
 Row(State='FPA_FPAC', count=1),
 Row(State='SRPAC_SRPA', count=1),
 Row(State='FSAEC_FSPAE', count=1),
 Row(State='FSPA_FSPAC', count=1),
 Row(State='FSRPAC_FSRPA', count=1),
 Row(State='SPA_', count=1),
 Row(State='SR_SRA', count=1),
 Row(State='SPA_RA', count=1),
 Row(State='PA_FRA', count=1),
 Row(State='SPAEC_SPA', count=1),
 Row(State='SRPAC_FSPA', count=1),
 Row(State='SA_SPA', count=1),
 Row(State='FSPA_', count=1),
 Row(State='FRA_PA', count=1),
 Row(State='FSRPA_FPA', count=1),
 Row(State='FSRAEC_FSPA', count=1),
 Row(State='RPAC_PA', count=1),
 Row(State='_FA', count=1),
 Row(State='FPAC_FPA', count=1),
 Row(State='FSRPAC_FSA', count=1),
 Row(State='FRPAC_FPA', count=1),
 Row(State='FS_RA', count=1),
 Row(State='FSA_', count=1),
 Row(State='SR_RPA', count=1),
 Row(State='SRPA_FRPA', count=1),
 Row(State='FPA_FS

In [24]:
df5 = df4.withColumn("State", when(df4["State"] !=  'CON' , "Others").otherwise(df4['State']))

In [25]:
df5.groupBy('State').count().orderBy('count').collect()

[Row(State='Others', count=418301), Row(State='CON', count=1470840)]

In [26]:
df5.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Dur: double (nullable = true)
 |-- Proto: string (nullable = true)
 |-- SrcAddr: string (nullable = true)
 |-- Sport: string (nullable = true)
 |-- Dir: string (nullable = true)
 |-- DstAddr: string (nullable = true)
 |-- Dport: string (nullable = true)
 |-- State: string (nullable = true)
 |-- sTos: double (nullable = true)
 |-- TotPkts: integer (nullable = true)
 |-- TotBytes: integer (nullable = true)
 |-- SrcBytes: integer (nullable = true)
 |-- Label: string (nullable = true)
 |-- Attacked: integer (nullable = true)



In [28]:
from pyspark.ml.feature import StringIndexer

#Convert Starttime to Double
indexer = StringIndexer(inputCol = 'StartTime', outputCol = 'StartTimeIndex')
outputFixed1 = indexer.fit(df5).transform(df5)
outputFixed1.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Dur: double (nullable = true)
 |-- Proto: string (nullable = true)
 |-- SrcAddr: string (nullable = true)
 |-- Sport: string (nullable = true)
 |-- Dir: string (nullable = true)
 |-- DstAddr: string (nullable = true)
 |-- Dport: string (nullable = true)
 |-- State: string (nullable = true)
 |-- sTos: double (nullable = true)
 |-- TotPkts: integer (nullable = true)
 |-- TotBytes: integer (nullable = true)
 |-- SrcBytes: integer (nullable = true)
 |-- Label: string (nullable = true)
 |-- Attacked: integer (nullable = true)
 |-- StartTimeIndex: double (nullable = false)



In [30]:
#Convert Proto to Double
indexer = StringIndexer(inputCol = 'Proto', outputCol = 'ProtoIndex')
outputFixed2 = indexer.fit(outputFixed1).transform(outputFixed1)
outputFixed2.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Dur: double (nullable = true)
 |-- Proto: string (nullable = true)
 |-- SrcAddr: string (nullable = true)
 |-- Sport: string (nullable = true)
 |-- Dir: string (nullable = true)
 |-- DstAddr: string (nullable = true)
 |-- Dport: string (nullable = true)
 |-- State: string (nullable = true)
 |-- sTos: double (nullable = true)
 |-- TotPkts: integer (nullable = true)
 |-- TotBytes: integer (nullable = true)
 |-- SrcBytes: integer (nullable = true)
 |-- Label: string (nullable = true)
 |-- Attacked: integer (nullable = true)
 |-- StartTimeIndex: double (nullable = false)
 |-- ProtoIndex: double (nullable = false)



In [32]:
#Convert feature Dir to Double
indexer = StringIndexer(inputCol = 'Dir', outputCol = 'DirIndex')
outputFixed3 = indexer.fit(outputFixed2).transform(outputFixed2)
outputFixed3.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Dur: double (nullable = true)
 |-- Proto: string (nullable = true)
 |-- SrcAddr: string (nullable = true)
 |-- Sport: string (nullable = true)
 |-- Dir: string (nullable = true)
 |-- DstAddr: string (nullable = true)
 |-- Dport: string (nullable = true)
 |-- State: string (nullable = true)
 |-- sTos: double (nullable = true)
 |-- TotPkts: integer (nullable = true)
 |-- TotBytes: integer (nullable = true)
 |-- SrcBytes: integer (nullable = true)
 |-- Label: string (nullable = true)
 |-- Attacked: integer (nullable = true)
 |-- StartTimeIndex: double (nullable = false)
 |-- ProtoIndex: double (nullable = false)
 |-- DirIndex: double (nullable = false)



In [34]:
#Convert State into Double
indexer = StringIndexer(inputCol = 'State', outputCol = 'StateIndex')
outputFixed4 = indexer.fit(outputFixed3).transform(outputFixed3)
outputFixed4.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Dur: double (nullable = true)
 |-- Proto: string (nullable = true)
 |-- SrcAddr: string (nullable = true)
 |-- Sport: string (nullable = true)
 |-- Dir: string (nullable = true)
 |-- DstAddr: string (nullable = true)
 |-- Dport: string (nullable = true)
 |-- State: string (nullable = true)
 |-- sTos: double (nullable = true)
 |-- TotPkts: integer (nullable = true)
 |-- TotBytes: integer (nullable = true)
 |-- SrcBytes: integer (nullable = true)
 |-- Label: string (nullable = true)
 |-- Attacked: integer (nullable = true)
 |-- StartTimeIndex: double (nullable = false)
 |-- ProtoIndex: double (nullable = false)
 |-- DirIndex: double (nullable = false)
 |-- StateIndex: double (nullable = false)



In [38]:
from pyspark.ml.feature import VectorAssembler

#Let us filter and take only the required fields
#assembler = VectorAssembler(inputCols = ['StartTime','Proto','Dir','State','sTos','TotPkts','TotBytes','SrcBytes'], outputCol = 'features')
assembler = VectorAssembler(inputCols = ['StartTimeIndex','ProtoIndex','DirIndex','StateIndex','sTos','TotPkts','TotBytes','SrcBytes'], outputCol = 'features')

In [40]:
output = assembler.transform(outputFixed4)
output.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Dur: double (nullable = true)
 |-- Proto: string (nullable = true)
 |-- SrcAddr: string (nullable = true)
 |-- Sport: string (nullable = true)
 |-- Dir: string (nullable = true)
 |-- DstAddr: string (nullable = true)
 |-- Dport: string (nullable = true)
 |-- State: string (nullable = true)
 |-- sTos: double (nullable = true)
 |-- TotPkts: integer (nullable = true)
 |-- TotBytes: integer (nullable = true)
 |-- SrcBytes: integer (nullable = true)
 |-- Label: string (nullable = true)
 |-- Attacked: integer (nullable = true)
 |-- StartTimeIndex: double (nullable = false)
 |-- ProtoIndex: double (nullable = false)
 |-- DirIndex: double (nullable = false)
 |-- StateIndex: double (nullable = false)
 |-- features: vector (nullable = true)



In [42]:
#Selct the final columns for model training and building
final_df = output.select('features','Attacked')

In [43]:
final_df.show(2)

+--------------------+--------+
|            features|Attacked|
+--------------------+--------+
|[1.0,1.0,1.0,1.0,...|       0|
|[1.0,1.0,1.0,1.0,...|       0|
+--------------------+--------+
only showing top 2 rows



In [44]:
#split into train/test dataset
train, test = final_df.randomSplit([0.7, 0.3])

In [45]:
#Apply multiple classification algorithm on the training set and compare the result in the test dataset

from pyspark.ml.classification import (DecisionTreeClassifier, RandomForestClassifier,GBTClassifier)
from pyspark.ml import Pipeline

dt = DecisionTreeClassifier(labelCol = 'Attacked', featuresCol = 'features')
rf = RandomForestClassifier(labelCol = 'Attacked', featuresCol = 'features')
gb = GBTClassifier(labelCol = 'Attacked', featuresCol = 'features')

In [46]:
dt_model = dt.fit(train)
rf_model = rf.fit(train)
gb_model = gb.fit(train)

In [47]:
dt_predictions = dt_model.transform(test)
rf_predictions = rf_model.transform(test)
gb_predictions = gb_model.transform(test)

In [48]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

binary_evaluator = BinaryClassificationEvaluator(labelCol = 'Attacked')
print('Decision Tree:', binary_evaluator.evaluate(dt_predictions))

Decision Tree: 0.8208530437840211


In [49]:
print('Random Forest:' , binary_evaluator.evaluate(rf_predictions))

Random Forest: 0.9309475425857097


In [50]:
print('Gradient Boosting:' , binary_evaluator.evaluate(gb_predictions))

Gradient Boosting: 0.9833524246668374


### KMeans clustering

In [58]:
final_df.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Attacked: integer (nullable = true)



In [59]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol = 'features', outputCol = 'scaledFeatures')
scaler_model = scaler.fit(final_df)
final_df_cluster = scaler_model.transform(final_df)
final_df_cluster.show(3)

+--------------------+--------+--------------------+
|            features|Attacked|      scaledFeatures|
+--------------------+--------+--------------------+
|[1.0,1.0,1.0,1.0,...|       0|[0.20634591038758...|
|[1.0,1.0,1.0,1.0,...|       0|[0.20634591038758...|
|[1.0,1.0,1.0,1.0,...|       0|[0.20634591038758...|
+--------------------+--------+--------------------+
only showing top 3 rows



In [61]:
from pyspark.ml.clustering import KMeans

In [62]:
kmeans = KMeans(featuresCol = 'scaledFeatures', k=8)
model = kmeans.fit(final_df_cluster)

In [63]:
print('WSSSE:', model.computeCost(final_df_cluster))

WSSSE: 2717955.0321508106


In [64]:
centers = model.clusterCenters()
print(centers)

[array([1.3782426 , 2.15872981, 2.32598973, 2.39111859, 0.00327757,
       0.03472349, 0.02610409, 0.01147578]), array([5.00240343e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       3.98554605e-03, 2.02094908e-03, 3.51419535e-04, 4.10647677e-04]), array([  1.23807546,   2.28878517,   3.11372052,   2.40844699,
         0.        , 141.97806282, 105.94839732, 293.76194535]), array([1.65076728e+00, 0.00000000e+00, 2.29432038e+00, 2.40844699e+00,
       3.30555122e+02, 6.03979894e-02, 1.57614170e-02, 4.52862470e-02]), array([  2.0634591 ,   2.28878517,   2.29432038,   2.40844699,
         0.        , 573.54567957, 719.81145651,  24.29826636]), array([1.71834129e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       4.38010884e-03, 1.71797450e-03, 3.09505716e-04, 3.25878820e-04]), array([2.88344872e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       2.95316043e-03, 1.67392841e-03, 3.05569236e-04, 3.15884812e-04]), array([  1.23807546,   2.28878517,   2.38256347,   2.40

In [68]:
model.transform(final_df_cluster).select('scaledFeatures', 'prediction').show(10)

+--------------------+----------+
|      scaledFeatures|prediction|
+--------------------+----------+
|[0.20634591038758...|         0|
|[0.20634591038758...|         0|
|[0.20634591038758...|         0|
|[0.20634591038758...|         0|
|[0.20634591038758...|         0|
|[0.20634591038758...|         0|
|[0.20634591038758...|         0|
|[0.20634591038758...|         0|
|[0.20634591038758...|         0|
|[0.20634591038758...|         0|
+--------------------+----------+
only showing top 10 rows



In [66]:
model.transform(final_df_cluster).show(2)

+--------------------+--------+--------------------+----------+
|            features|Attacked|      scaledFeatures|prediction|
+--------------------+--------+--------------------+----------+
|[1.0,1.0,1.0,1.0,...|       0|[0.20634591038758...|         0|
|[1.0,1.0,1.0,1.0,...|       0|[0.20634591038758...|         0|
+--------------------+--------+--------------------+----------+
only showing top 2 rows

